# Node.js Streams homework

### Reference
* Short intro https://nodejs.dev/learn/nodejs-streams
* Node.js docs https://nodejs.org/api/stream.html

### Warning

If you encounter ```SyntaxError: Identifier <some-variable-name> has already been declared```
go to Kernel -> Restart in order to clear all declared variables.

### Task 1
You will have to apply the concepts presented at class in order to convert an audio file from a format to another without reading it entirely into memory.

This process is generically called transcoding, although sometimes transmuxing is a more accurate term. You will understand the difference after you solve task 2.

Since an image is worth a thousand words, here is what you'll actually do (in a streamed fashion):

![title](../img/transcoding.png)

### Solution

Please, don't change anything in the ```DON'T CHANGE``` sections of the code.

You can test that the conversion is successfully done by accessing http://localhost:5000/, where you have an audio player.

I suggest opening the url in an incognito session to prevent browser caching.

In [ ]:
'use strict';

///////////// DON'T CHANGE /////////////
// check docs for https://www.npmjs.com/package/prism-media
const prism = require('prism-media');
///////////////////////////////////////

// You can add your dependencies here. 
// Note that you should use only Node.js builtin modules.
// aka no third-party dependencies allowed, like the one above.
const fstream = require('fs');
///////////// DON'T CHANGE /////////////
const SOURCE_FILE = '../data/weekend.mp3';
const DESTINATION_FILE = '../www/static/audio.ogg';

const OUT_MUXER = 'ogg';
const OUT_SAMPLE_RATE = '48000';
const OUT_NUM_CHANNELS = '2';

const transcoder = new prism.FFmpeg({
  args: [
    '-analyzeduration', '0',
    '-loglevel', '0',
    '-f', OUT_MUXER,
    '-ar', OUT_SAMPLE_RATE,
    '-ac', OUT_NUM_CHANNELS,
  ],
});
///////////////////////////////////////

// You can add your logic here.
// Note that you should use only Node.js Stream APIs.
// aka reading of the whole file in memory and afterwards converting it is not scored.
fstream.createReadStream(SOURCE_FILE)
  .pipe(transcoder)
  .pipe(fstream.createWriteStream(DESTINATION_FILE));

### Task 2

Use the following code to find out what codec is used by the source file and by the destination file, respectively.

Also answer to the following questions:
1. What's the difference between container and codec?
2. What format do you think is more appropriate for live streaming conferences? (the one used by the source file, or the one used by the destionation file)

In [1]:
const { spawn } = require('child_process');
const ffprobe = require('ffprobe-static');
const SOURCE_FILE1= '../data/weekend.mp3';
const DESTINATION_FILE1 = '../www/static/audio.ogg';

const command = ffprobe.path;
console.log(command);

{
const args = [
    '-v', 'error',
    '-select_streams', 'a:0',
    '-show_entries', 'stream=codec_long_name',
    '-of', 'default=noprint_wrappers=1:nokey=1',
    //path_to_file
    //process.argv[2]
    //SOURCE_FILE1
   DESTINATION_FILE1,
    
];

const subprocess = spawn(command, args);
subprocess.on('exit', (code, signal) => {

    if (code !== null) {

        if (code === 0) {
            console.log('Process exit with succes');
            return;
        }

        console.error(`Process exit code ${code}`);
        return;
    }

    if (signal !== null) {
        console.error(`Process killed with signal ${signal}`);
    }
    
});

subprocess.stdout.on('data', (data) => {
  console.log(`stdout: ${data}`);
})
}


/usr/project/hw1/node_modules/ffprobe-static/bin/linux/x64/ffprobe


<ref *1> Socket {
  connecting: false,
  _hadError: false,
  _parent: null,
  _host: null,
  _readableState: ReadableState {
    objectMode: false,
    highWaterMark: 16384,
    buffer: BufferList { head: null, tail: null, length: 0 },
    length: 0,
    pipes: [],
    flowing: true,
    ended: false,
    endEmitted: false,
    reading: true,
    constructed: true,
    sync: false,
    needReadable: true,
    emittedReadable: false,
    readableListening: false,
    resumeScheduled: true,
    errorEmitted: false,
    emitClose: false,
    autoDestroy: true,
    destroyed: false,
    errored: null,
    closed: false,
    closeEmitted: false,
    defaultEncoding: 'utf8',
    awaitDrainWriters: null,
    multiAwaitDrain: false,
    readingMore: false,
    decoder: null,
    encoding: null,
    [Symbol(kPaused)]: false
  },
  _events: [Object: null prototype] {
    end: [Function: onReadableStreamEnd],
    close: [Function (anonymous)],
    data: [Function (anonymous)]
  },
  _eventsCount:

stdout: Vorbis

Process exit with succes


In [ ]:
/*  stdout: MP3 (MPEG audio layer 3) - in argv used const SOURCE_FILE1= '../data/weekend.mp3'
    stdout: Vorbis  - in argv used const DESTINATION_FILE1 = '../www/static/audio.ogg'
    There are some errors I could not fix, but the result is at the end of the output. Any feedback would be grate
    for fixing the errors encountered.
*/


/* 1.A codec is a method for encoding and decoding data and more specifically, a protocol for compressing 
data, especially video. Selecting the right codec can depend on several factors.Target file size, output quality,
and delivery method all factor in. Currently h.264 is the predominant codec on the web.

In comparison, a container is what holds the grouping of compressed video as defined by the codec. A container is 
also referred to as a format. The container takes care of packaging, transport, and presentation. The container is
usually represented by a file extension. For example the MPEG4 container is usually represented by a .mp4 file 
type.
*/

/* 2.The difference between OGG Vorbis and MP3 format is that OGG is an open-source format that supports two to 256
channels with better sound quality and smaller compressed file while MP3 is an open-source format that can support two discrete channels,
it also supports joint-stereo, however, is has inferior sound quality.
    In conclusion, OGG Vorbis is the right choice for  live streaming conferences. 
*/
